In [1]:
import ants
import os
import pandas as pd
from tqdm import tqdm

### Load atlas and labels

In [2]:
atlas = ants.image_read('../data/WHS_SD_rat_atlas_v4_pack/WHS_SD_rat_atlas_v4_resized_L_v_R.nii.gz')
labels = pd.read_csv('../data/WHS_SD_rat_atlas_v4_pack/Labels_split.csv')

In [3]:
region_dict = pd.Series(labels['Region'].values, index=labels['Label']).to_dict()
voxel_volume = (0.003906*0.003906*0.003906) #WHS in cm

### Calculate stats for each animal

In [4]:
tissue_path = '../data/registered_tissues/GM_modulated'
out_path = os.path.join(tissue_path, 'stats_csv')
os.makedirs(out_path, exist_ok = True)

In [6]:
tissue_files = [f.name for f in os.scandir(tissue_path) if f.is_file()]
pbar = tqdm(total = len(tissue_files))

for file in tissue_files:
    file_path = os.path.join(tissue_path, file)
    img = ants.image_read(file_path)
    stats = ants.label_stats(img, atlas)
    regions = stats['LabelValue']
    avg = stats['Mean']
    sd = np.sqrt(stats['Variance'])
    vol = stats['Mass']*voxel_volume
    region_names = [region_dict[int(i)] for i in regions]
    results = pd.DataFrame({
        'RegionNumber': regions,
        'RegionName': region_names,
        'average': avg,
        'sd': sd,
        'volume': vol
    }, index = None)
    results.to_csv(os.path.join(out_path, file.split('.')[0]) + '_stats.csv')
    pbar.update(1)
pbar.close

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:40<00:00, 10.08s/it]

<bound method tqdm.close of <tqdm.std.tqdm object at 0x7f0f0a2741a0>>